In [176]:
# from pymatgen.ext.matproj import MPRester
# from pymatgen.core import Structure
# from pymatgen.core.surface import SlabGenerator

from mp_api.client import MPRester
import crystal_toolkit

from pymatgen.transformations.advanced_transformations import SlabTransformation
from pymatgen.core import Structure, Lattice

In [116]:
from pymatgen.io.vasp import Poscar

In [117]:
def save_cif(structure, name_cife):
    cif_filename = f"{name_cife}.cif"
    structure.to(fmt="cif", filename=cif_filename)

    print(f"CIF file written as {cif_filename}")

In [133]:
def orthogonalize_structure(structure_pymat):
    # Create new lattice with 90° angles
    new_structure_pymat = Lattice.from_parameters(
        a=structure_pymat.lattice.a,
        b=structure_pymat.lattice.b,
        c=structure_pymat.lattice.c,
        alpha=90,  # Modified angle
        beta=90,   # Modified angle
        gamma=90   # Modified angle
    )

    # Update structure with new lattice while keeping fractional coordinates
    structure_ortho_pymat = Structure(
        lattice=new_structure_pymat,
        species=structure_pymat.species,
        coords=structure_pymat.frac_coords,
        coords_are_cartesian=False
    )

    structure_ortho_ase = structure_ortho_pymat.to_ase_atoms()

    return structure_ortho_ase, structure_ortho_pymat

# Create Slab and Supercell

## Call cathode materials from Materials Project

In [118]:
api_key = "DYEm5SmJXXuDYcbXQPwQReoTqyme8WbK"  # Add your Materials Project API key here

mp_id = "mp-25411"

with MPRester(api_key) as mpr:
    # structure = mpr.get_structure_by_material_id(mp_id)
    docs = mpr.materials.summary.search(material_ids=[mp_id], fields=["structure"])

    structure = docs[0].structure

    # origin_data = structure[0].origins
    # print(origin_data)

# primitive_cell = structure.get_primitive_structure()
# primitive_cell.to("structure.cif")

print(structure)

Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 7096.96it/s]

Full Formula (Li1 Ni1 O2)
Reduced Formula: LiNiO2
abc   :   4.969023   4.969024   4.969022
angles:  33.418400  33.418392  33.418404
pbc   :       True       True       True
Sites (4)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  Li    2e-06     2e-06     2e-06        0.004
  1  Ni    0.500001  0.500001  0.500001     0.959
  2  O     0.759016  0.759016  0.759016     0.006
  3  O     0.240982  0.240982  0.240982     0.006


In [125]:
structure

In [120]:
# Write the structure to a CIF file
save_cif(structure, f"{mp_id}")

CIF file written as mp-25411.cif


## Create a standardized structure

In [121]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [122]:
analyzer = SpacegroupAnalyzer(structure) 
# try with symprec = 0.1 (looser one for materials project)
structure_standardized_mp = analyzer.get_conventional_standard_structure()

print(structure_standardized_mp)

Full Formula (Li3 Ni3 O6)
Reduced Formula: LiNiO2
abc   :   2.857331   2.857331  14.061569
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (12)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Li    0         0         0
  1  Li    0.666667  0.333333  0.333333
  2  Li    0.333333  0.666667  0.666667
  3  Ni    0.333333  0.666667  0.166667
  4  Ni    1         1         0.5
  5  Ni    0.666667  0.333333  0.833333
  6  O     0.666667  0.333333  0.092349
  7  O     0         0         0.240984
  8  O     0.333333  0.666667  0.425683
  9  O     0.666667  0.333333  0.574317
 10  O     0         0         0.759016
 11  O     0.333333  0.666667  0.907651


In [123]:
structure_standardized_mp

In [124]:
# Write the structure to a CIF file
save_cif(structure_standardized_mp, f"{mp_id}_std")
# cif_filename_std = f"{mp_id}_std.cif"
# structure_standardized.to(fmt="cif", filename=cif_filename_std)

# print(f"CIF file standardized and saved as {cif_filename_std}")

CIF file written as mp-25411_std.cif


## Read POSCAR used by Alex'

In [177]:
# Read a POSCAR and write to a CIF.
structure_alex = Structure.from_file("quantumatk/POSCAR_mp-25411_alex")
# structure.to(filename="CsCl.cif")
structure_alex

In [128]:
print(structure_alex)

Full Formula (Li3 Ni3 O6)
Reduced Formula: LiNiO2
abc   :   2.905345   2.905346  14.401854
angles:  90.000000  90.000000 120.000004
pbc   :       True       True       True
Sites (12)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Li    0.333333  0.666667  0.166667
  1  Li    0         0         0.5
  2  Li    0.666667  0.333333  0.833333
  3  Ni    0         0         0
  4  Ni    0.666667  0.333333  0.333333
  5  Ni    0.333333  0.666667  0.666667
  6  O     0.666667  0.333333  0.075577
  7  O     0         0         0.257756
  8  O     0.333333  0.666667  0.40891
  9  O     0.666667  0.333333  0.59109
 10  O     0         0         0.742244
 11  O     0.333333  0.666667  0.924423


In [144]:
structure_alex_ortho_ase, structure_alex_ortho_pymat = orthogonalize_structure(structure_alex)
structure_alex_ortho_pymat

In [145]:
print(structure_alex_ortho_pymat)

Full Formula (Li3 Ni3 O6)
Reduced Formula: LiNiO2
abc   :   2.905345   2.905346  14.401854
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (12)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Li    0.333333  0.666667  0.166667
  1  Li    0         0         0.5
  2  Li    0.666667  0.333333  0.833333
  3  Ni    0         0         0
  4  Ni    0.666667  0.333333  0.333333
  5  Ni    0.333333  0.666667  0.666667
  6  O     0.666667  0.333333  0.075577
  7  O     0         0         0.257756
  8  O     0.333333  0.666667  0.40891
  9  O     0.666667  0.333333  0.59109
 10  O     0         0         0.742244
 11  O     0.333333  0.666667  0.924423


## Create a slab using ASE

In [168]:
structure_standardized = structure_alex

In [169]:
# Compute the plane spacing from the lattice and miller index
from pymatgen.core.surface import get_symmetrically_distinct_miller_indices
# from pymatgen.analysis.surface_analysis import get_spacing

import ase as ase
from ase.build import surface, bulk, cut, make_supercell, add_vacuum
from pymatgen.io.ase import AseAtomsAdaptor

# structure = structure_standardized
structure_standardized_ase = AseAtomsAdaptor.get_atoms(structure_standardized)   # convert the pymatgen structure to an ASE object
# length = 20

### Create using cut()

In [170]:
# # Cut with a and b aligned differently
# slab_cut_ase = cut(
#     atoms=structure_standardized_ase,
#     a=(0, 1, 0),  # y becomes a-axis
#     b=(1, 0, 0),  # x becomes b-axis
#     c=(1, 0, 4),  # (104) direction along z
#     nlayers=3,
#     origo=(0, 0, 0),
#     extend=1.0
# )

# slab_cut_pymat = AseAtomsAdaptor.get_structure(slab_cut_ase)
# slab_cut_pymat

### Automatically add 20 \AA (in total) while creating the slab using $surface()$

In [173]:
slab_10_ase = surface(structure_standardized_ase,(1,0,4),layers=2,vacuum=10)

slab_10_pymat = AseAtomsAdaptor.get_structure(slab_10_ase)
slab_10_pymat

In [174]:
slab_10_sorted_pymat = slab_10_pymat.get_sorted_structure()
print(slab_10_sorted_pymat)
save_cif(slab_10_sorted_pymat, f"{mp_id}_slab_10_sorted_noorthoposcar_2lay")
poscar = Poscar(slab_10_sorted_pymat)
poscar.write_file(f"POSCAR_{mp_id}_slab_10_sorted_noorthoposcar_2lay")

Full Formula (Li6 Ni6 O12)
Reduced Formula: LiNiO2
abc   :  18.505942   2.905346  24.124812
angles:  90.000000  90.000000 108.299885
pbc   :       True       True      False
Sites (24)
  #  SP           a         b         c    tags
---  ----  --------  --------  --------  ------
  0  Li    0.833333  0.666667  0.414511       2
  1  Li    0.5       0         0.414511       2
  2  Li    0.248697  0.997393  0.5            2
  3  Li    0.915363  0.330726  0.5            1
  4  Li    0.58203   0.66406   0.5            1
  5  Li    0.330727  0.661453  0.585489       1
  6  Ni    0         0         0.414511       2
  7  Ni    0.748697  0.997393  0.5            2
  8  Ni    0.333333  0.666667  0.414511       2
  9  Ni    0.08203   0.66406   0.5            1
 10  Ni    0.830727  0.661453  0.585489       1
 11  Ni    0.415363  0.330726  0.5            1
 12  O     0.003908  0.007815  0.497348       2
 13  O     0.744789  0.989578  0.417163       2
 14  O     0.670575  0.34115   0.497348       2

#### Structure to sit on top of the vacuum

In [150]:
slab_10_top_vacuum_ase = slab_10_ase.copy()

# Move the slab so it sits at the bottom of the cell (i.e., vacuum is on top)
positions = slab_10_ase.get_positions()
z_positions = positions[:, 2]
z_shift = -min(z_positions)  # Shift so the lowest atom is at z = 0
positions[:, 2] += z_shift
slab_10_top_vacuum_ase.set_positions(positions)

# # Optional: check or print the final z-range
# z_min, z_max = slab_10_ase.get_positions()[:, 2].min(), slab_10_ase.get_positions()[:, 2].max()
# print(f"Slab z-range: {z_min:.2f} Å to {z_max:.2f} Å")

slab_10_top_vacuum_pymat = AseAtomsAdaptor.get_structure(slab_10_top_vacuum_ase)
slab_10_top_vacuum_pymat

#### Orthogonalize slab

In [151]:
# slab_10_top_vacuum_ortho_ase, slab_10_top_vacuum_ortho_pymat = orthogonalize_structure(slab_10_top_vacuum_pymat)
# slab_10_top_vacuum_ortho_pymat
# print("Note: orthogonalizing removes the tags of layers")

#### Create supercell

In [156]:
sc_10_top_vacuum_ortho_ase = slab_10_top_vacuum_ase.repeat((1, 3, 1))

sc_10_top_vacuum_ortho_pymat = AseAtomsAdaptor.get_structure(sc_10_top_vacuum_ortho_ase)
sc_10_top_vacuum_ortho_pymat

#### Manually swap so that x-side becomes y-side as in Quantumatk

In [157]:
sc_10_top_vacuum_ortho_rotated_ase = sc_10_top_vacuum_ortho_ase.copy()

# Swap a and b vectors
cell = sc_10_top_vacuum_ortho_rotated_ase.cell
sc_10_top_vacuum_ortho_rotated_ase.set_cell([cell[1], -cell[0], cell[2]], scale_atoms=True)

sc_10_top_vacuum_ortho_rotated_pymat = AseAtomsAdaptor.get_structure(sc_10_top_vacuum_ortho_rotated_ase)
sc_10_top_vacuum_ortho_rotated_pymat

In [158]:
sc_10_top_vacuum_ortho_rotated_sorted_pymat = sc_10_top_vacuum_ortho_rotated_pymat.get_sorted_structure()
print(sc_10_top_vacuum_ortho_rotated_sorted_pymat)
save_cif(sc_10_top_vacuum_ortho_rotated_sorted_pymat, f"{mp_id}_sc_10_top_vacuum_ortho_rotated_sorted")
poscar = Poscar(sc_10_top_vacuum_ortho_rotated_sorted_pymat)
poscar.write_file(f"POSCAR_{mp_id}_sc_10_top_vacuum_ortho_rotated_sorted")

Full Formula (Li27 Ni27 O54)
Reduced Formula: LiNiO2
abc   :   8.716037  18.505942  26.783066
angles:  90.000000  90.000000  90.000000
pbc   :       True       True      False
Sites (108)
  #  SP           a         b         c    tags
---  ----  --------  --------  --------  ------
  0  Li    0.833333  0.222222  0              3
  1  Li    0.5       0         0              3
  2  Li    0.265257  0.111111  0.08442        3
  3  Li    0.931923  0.222222  0.08442        2
  4  Li    0.59859   0         0.08442        2
  5  Li    0.363847  0.111111  0.16884        2
  6  Li    0.030513  0.222222  0.16884        1
  7  Li    0.69718   0         0.16884        1
  8  Li    0.462437  0.111111  0.25326        1
  9  Li    0.833333  0.555556  0              3
 10  Li    0.5       0.333333  0              3
 11  Li    0.265257  0.444444  0.08442        3
 12  Li    0.931923  0.555556  0.08442        2
 13  Li    0.59859   0.333333  0.08442        2
 14  Li    0.363847  0.444444  0.16884      

#### Flip in the z coordinate

In [159]:
def flip_structure(structure_ase, axis):

    if axis == "z":
        id = 2
    elif axis == "y":
        id = 1
    elif axis == "z":
        id = 0

    structure_flip_ase = structure_ase.copy()

    coor_max = structure_ase.positions[:, id].max()
    structure_flip_ase.positions[:, id] = coor_max - structure_flip_ase.positions[:, id]

    structure_flip_pymat = AseAtomsAdaptor.get_structure(structure_flip_ase)
    structure_flip_pymat

    return structure_flip_ase, structure_flip_pymat

In [160]:
sc_10_top_vacuum_ortho_rotated_flip_ase, sc_10_top_vacuum_ortho_rotated_flip_pymat = flip_structure(sc_10_top_vacuum_ortho_rotated_ase, axis="z")
sc_10_top_vacuum_ortho_rotated_flip_pymat

In [161]:
sc_10_top_vacuum_ortho_rotated_sorted_pymat = sc_10_top_vacuum_ortho_rotated_pymat.get_sorted_structure()
print(sc_10_top_vacuum_ortho_rotated_sorted_pymat)
save_cif(sc_10_top_vacuum_ortho_rotated_sorted_pymat, f"{mp_id}_sc_10_top_vacuum_ortho_rotated_sorted_flip")
poscar = Poscar(sc_10_top_vacuum_ortho_rotated_sorted_pymat)
poscar.write_file(f"POSCAR_{mp_id}_sc_10_top_vacuum_ortho_rotated_sorted_flip")

Full Formula (Li27 Ni27 O54)
Reduced Formula: LiNiO2
abc   :   8.716037  18.505942  26.783066
angles:  90.000000  90.000000  90.000000
pbc   :       True       True      False
Sites (108)
  #  SP           a         b         c    tags
---  ----  --------  --------  --------  ------
  0  Li    0.833333  0.222222  0              3
  1  Li    0.5       0         0              3
  2  Li    0.265257  0.111111  0.08442        3
  3  Li    0.931923  0.222222  0.08442        2
  4  Li    0.59859   0         0.08442        2
  5  Li    0.363847  0.111111  0.16884        2
  6  Li    0.030513  0.222222  0.16884        1
  7  Li    0.69718   0         0.16884        1
  8  Li    0.462437  0.111111  0.25326        1
  9  Li    0.833333  0.555556  0              3
 10  Li    0.5       0.333333  0              3
 11  Li    0.265257  0.444444  0.08442        3
 12  Li    0.931923  0.555556  0.08442        2
 13  Li    0.59859   0.333333  0.08442        2
 14  Li    0.363847  0.444444  0.16884      

In [162]:
print("Note: why no difference between before and after being flipped")

Note: why no difference between before and after being flipped


### Creating the slab using $surface()$

In [ ]:
slab_04_ase = surface(structure_standardized_ase,(1,0,4),layers=3,vacuum=0.4)

slab_04_pymat = AseAtomsAdaptor.get_structure(slab_04_ase)
slab_04_pymat

In [ ]:
# print(slab_04_pymat)

### Add vacuum

In [ ]:
slab_04_vacuum_ase = slab_04_ase.copy()

add_vacuum(slab_04_vacuum_ase, 20.0) 

slab_04_vacuum_pymat = AseAtomsAdaptor.get_structure(slab_04_vacuum_ase)
slab_04_vacuum_pymat

In [ ]:
# print(slab_04_flip_vacuum_pymat)

### Flip the slab in the z-coordinate

In [ ]:
slab_04_vacuum_flip_ase, slab_04_vacuum_flip_pymat = flip_structure(slab_04_vacuum_ase, axis="z")
slab_04_vacuum_flip_pymat

### Read angle (orthogonality) of slab

In [ ]:
# Check original slab cell angles (skewed!)
from ase.geometry import cell_to_cellpar
print("Angles:", cell_to_cellpar(slab_04_vacuum_flip_ase.cell)[3:])

## Create supercell

In [ ]:
# supercell_ase = slab_04_flip_vacuum_ase.copy()

supercell_ase = slab_04_vacuum_flip_ase.repeat((1, 3, 1))

supercell_pymat = AseAtomsAdaptor.get_structure(supercell_ase)
supercell_pymat

In [ ]:
print("Angles:", cell_to_cellpar(supercell_ase.cell)[3:])

### Rotate so that the x-direction (long side) becomes y-direction as in QuantumATK

#### Method using $supercell\_ase.rotate()$

In [ ]:
# supercell_rotated_a_ase = supercell_ase.copy()

# # Rotate around z-axis by 90°: x → y
# supercell_rotated_a_ase.rotate(90, 'z', rotate_cell=True)

# # # Optional: center supercell_rotated_a_ase and reset origin
# # supercell_rotated_a_ase.positions[:, 2] -= supercell_rotated_a_ase.positions[:, 2].min()

# supercell_rotated_a_pymat = AseAtomsAdaptor.get_structure(supercell_rotated_a_ase)
# supercell_rotated_a_pymat

print("Note: doesn't work")

#### Method using $rotate()$

In [ ]:
# from ase.build import rotate

# supercell_rotated_b_ase = supercell_ase.copy()

# # # Optional: center supercell_rotated_b_ase and reset origin
# # supercell_rotated_b_ase.positions[:, 2] -= supercell_rotated_b_ase.positions[:, 2].min()

# # Rotate around z-axis (in-plane rotation)
# rotate(supercell_rotated_b_ase, 'z', 90, rotate_cell=True)

# supercell_rotated_b_pymat = AseAtomsAdaptor.get_structure(supercell_rotated_b_ase)
# supercell_rotated_b_pymat

print("Note: TypeError: rotate() missing 2 required positional arguments: 'b1' and 'b2'")

#### Method manually swap cell axes

In [ ]:
supercell_rotated_c_ase = supercell_ase.copy()

# Swap a and b vectors
cell = supercell_rotated_c_ase.cell
supercell_rotated_c_ase.set_cell([cell[1], -cell[0], cell[2]], scale_atoms=True)

supercell_rotated_c_pymat = AseAtomsAdaptor.get_structure(supercell_rotated_c_ase)
supercell_rotated_c_pymat

In [ ]:
# print(supercell_rotated_c_pymat)

### Modify to orthogonal 

In [ ]:
supercell_ortho_pymat = orthogonalize_structure(supercell_rotated_c_pymat)
supercell_ortho_pymat

In [ ]:
print(supercell_ortho_pymat)
save_cif(supercell_ortho_pymat, f"{mp_id}_sc_ortho")
poscar = Poscar(supercell_ortho_pymat)
poscar.write_file(f"POSCAR_{mp_id}_sc_ortho")

In [ ]:
supercell_ortho_sorted_pymat = supercell_ortho_pymat.get_sorted_structure()
print(supercell_ortho_sorted_pymat)
save_cif(supercell_ortho_sorted_pymat, f"{mp_id}_sc_ortho_sorted")
poscar = Poscar(supercell_ortho_sorted_pymat)
poscar.write_file(f"POSCAR_{mp_id}_sc_ortho_sorted")

### Cut along Miller index (FAILED)

In [ ]:
ase_slab_cut = cut(ase_slab, b=(1, 0, 4), clength=0.4, nlayers=3)

In [ ]:
ase_slab_cut_pymat = AseAtomsAdaptor.get_structure(ase_slab_cut)
ase_slab_cut_pymat

### Add vacuum on top

In [ ]:
z_max = ase_slab.positions[:, 2].max()
new_c = ase_slab.get_cell()
new_c[2, 2] = z_max + 20.0
ase_slab.set_cell(new_c, scale_atoms=False)

In [ ]:
ase_surface

In [ ]:
ase_surface.get_positions()

In [ ]:
# from ase import Atoms
# from ase.build import fcc111, bulk
# from ase.visualize import view

# # Build a Pt(111) slab
# slab = fcc111('Pt', size=(4,4,4), vacuum=10.0)

# # Build a molecule (example)
# molecule = bulk('CO', 'mx2')

# # Align and add the molecule on top of the slab
# slab_top = molecule.copy()
ase_surface.translate((0,0,10))
# ase_surface.extend(slab_top)

print(ase_surface)

In [ ]:
ase_structure_pymat = AseAtomsAdaptor.get_structure(ase_surface)
ase_structure_pymat

In [ ]:
print(ase_structure_pymat)

## Create supercell

In [ ]:
sc_20_ase = slab_20_ase.repeat((3,1,1))
sc_20_pymat = AseAtomsAdaptor.get_structure(sc_20_ase)
sc_20_pymat

In [ ]:
# Get the z-coordinates of all atoms
z_coords = ase_surface.get_positions()[:, 2]

# Find the minimum z-coordinate (bottom layer)
min_z = min(z_coords)

In [ ]:
# Calculate the desired z-coordinate (e.g., set the bottom to z=0)
target_z = 0

# Calculate the shift required
shift_z = target_z - min_z

# Move all atoms down
ase_surface.positions[:, 2] += shift_z

In [ ]:
ase_structure_pymat = AseAtomsAdaptor.get_structure(ase_surface)
ase_structure_pymat

In [ ]:
print(ase_structure_pymat)

In [ ]:
z_coords

In [ ]:
ase_surface.get_cell()

In [ ]:
slab.positions

In [ ]:
ase_surface.center(vacuum=5, axis=2)

ase_structure_pymat = AseAtomsAdaptor.get_structure(ase_surface)
ase_structure_pymat

In [ ]:
# from ase.visualize import view
# ase_surface.edit()
# view(ase_surface)

In [ ]:
ase_surface.top

In [ ]:
ase_surface.set_cell([structure.lattice.a,structure.lattice.b,structure.lattice.c+length]) # make the vacuum gap

ase_surface

newstructure = AseAtomsAdaptor.get_structure(ase_surface)  # convert the structure back to a pymatgen object

newstructure

In [ ]:
sc_structure = ase_surface.repeat((3,1,1))
sc_structure_pymatgen = AseAtomsAdaptor.get_structure(sc_structure)
sc_structure_pymatgen

In [ ]:
save_cif(sc_structure_pymatgen, f"{mp_id}_sc_ase")

## Create a slab using Pymatgen 

In [ ]:
# Define slab parameters
miller_index = [1, 0, 4]  # Miller index for the slab
min_slab_size = 3      # Thickness of the slab
min_vacuum_size = 4    # Thickness of the vacuum layer
# tol = 0.01

# Create the transformation with user parameters
slab_transformation = SlabTransformation(
    miller_index=miller_index,
    min_slab_size=min_slab_size,
    min_vacuum_size=min_vacuum_size,
    lll_reduce=True,  # makes the slab more orthogonal 
    center_slab=False,  # centers the slab with vacuum on top and bottom
    in_unit_planes=True  # means slab thicknesses are specified as number of planes, not Angstroms
    # tol = tol
)

# Apply the transformation
slab_7layer_pymat = slab_transformation.apply_transformation(structure_standardized)
slab_7layer_pymat

In [ ]:
print(structure_slab)

In [ ]:
# spacing = get_spacing(structure_standardized.lattice, miller_index)
# thickness = spacing * min_slab_size

In [ ]:
# # Manually add vacuum of 10 Å
# slab_thickness = structure_slab.get_slab_thickness()
# vacuum = 10.0
# total_c = slab_thickness + vacuum
# structure_slab.lattice = structure_slab.lattice.copy(a=structure_slab.lattice.a, b=structure_slab.lattice.b, c=total_c)
# structure_slab.translate_sites(range(len(structure_slab)), [0, 0, vacuum / 2])

In [ ]:
print(structure_slab)

In [ ]:
save_cif(structure_slab, f"{mp_id}_slab")

# cif_filename_slab = f"{mp_id}_slab.cif"
# structure_slab.to(fmt="cif", filename=cif_filename_slab)

# print(f"CIF file created as slab as {cif_filename_slab}")

In [ ]:
from pymatgen.transformations.standard_transformations import SupercellTransformation

In [ ]:
# Expand the the surface plane (e.g., if you want to add an adsorbate later)
supercell_transformation = SupercellTransformation.from_scaling_factors(3, 3, 1)
structure_supercell = supercell_transformation.apply_transformation(structure_slab)
structure_supercell

In [ ]:
print(structure_supercell)

In [ ]:
save_cif(structure_supercell, f"{mp_id}_supercell")

In [ ]:
# Shift slab to bottom (vacuum on top only)

# Find the minimum z-coordinate of the atoms
z_coords = [site.coords[-1] for site in structure_supercell.sites]
min_z = min(z_coords)

# Shift all atoms so the slab starts at z = 0
structure_supercell_bottom = structure_supercell.translate_sites(
    indices=list(range(len(structure_supercell))),
    vector=[0, 0, -min_z]
)

In [ ]:
structure_supercell_bottom

In [ ]:
save_cif(structure_supercell_bottom, f"{mp_id}_supercell_bottom")

In [ ]:
# # Expand the the surface plane (e.g., if you want to add an adsorbate later)
# supercell_transformation_bottom = SupercellTransformation.from_scaling_factors(3, 1, 1)
# structure_supercell_bottom = supercell_transformation.apply_transformation(structure_slab_bottom)
# structure_supercell_bottom

In [ ]:
import random
from pymatgen.core import Element
from pymatgen.transformations.standard_transformations import SubstitutionTransformation

In [ ]:
# # Copy the supercell structure so we don’t modify the original
# nmc_structure = structure_supercell.copy()

# # Get all Ni sites (index + site object)
# ni_indices = [i for i, site in enumerate(nmc_structure) if site.specie.symbol == "Ni"]

# # Ensure we have enough Ni to substitute
# if len(ni_indices) < 3:
#     raise ValueError("Not enough Ni atoms to apply 1:1:1 Ni:Mn:Co substitution.")

# # Shuffle for randomness, or sort to keep it deterministic
# random.shuffle(ni_indices)

# # Calculate number of substitutions
# n_total = len(ni_indices)
# n_mn = n_co = n_total // 3
# n_ni = n_total - n_mn - n_co  # Keep remainder as Ni

# # Replace indices
# for idx in ni_indices[:n_mn]:
#     nmc_structure[idx] = Element("Mn")

# for idx in ni_indices[n_mn:n_mn + n_co]:
#     nmc_structure[idx] = Element("Co")

# # Done. nmc_structure is now Ni:Mn:Co = ~1:1:1

In [ ]:
from ase.io import read, write
from collections import Counter

In [ ]:
def substitute_Ni_atoms(structure, nmc_ratio, seed=None):
    """
    Replace Ni atoms in the structure with Mn and Co according to a given NMC ratio.

    Parameters:
        structure (ASE Atoms): Structure containing Ni atoms.
        nmc_ratio (tuple): A 3-tuple (Ni_fraction, Mn_fraction, Co_fraction), should sum to 1.
        seed (int): Optional seed for reproducibility.

    Returns:
        ASE Atoms object with substitutions.
    """
    nmc_structure = structure.copy()

    if seed is not None:
        random.seed(seed)

    # Find all Ni atoms
    ni_indices = [i for i, site in enumerate(nmc_structure) if site.specie.symbol == "Ni"]
    n_total = len(ni_indices)

    # Compute target numbers for each element
    ni_frac, mn_frac, co_frac = nmc_ratio
    n_ni = round(n_total * ni_frac)
    n_mn = round(n_total * mn_frac)
    n_co = n_total - n_ni - n_mn  # Adjust last one to ensure sum is correct

    random.shuffle(ni_indices)

    for i in ni_indices[:n_mn]:
        nmc_structure[i].symbol = "Mn"
    for i in ni_indices[n_mn:n_mn + n_co]:
        nmc_structure[i].symbol = "Co"
    # Remaining are Ni

    print(f"NMC ratio: {nmc_ratio}")
    nmc_structure

    return nmc_structure

In [ ]:
structure = structure_supercell_bottom
seed = 42
nmc_ratio = (1/3, 1/3, 1/3)

In [ ]:
nmc_structure = structure.copy()

if seed is not None:
    random.seed(seed)

# Find all Ni atoms
ni_indices = [i for i, site in enumerate(nmc_structure) if site.specie.symbol == "Ni"]
print(ni_indices)
n_total = len(ni_indices)

# Compute target numbers for each element
ni_frac, mn_frac, co_frac = nmc_ratio
n_ni = round(n_total * ni_frac)
n_mn = round(n_total * mn_frac)
n_co = n_total - n_ni - n_mn  # Adjust last one to ensure sum is correct

random.shuffle(ni_indices)
print(ni_indices)

for i in ni_indices[n_ni:n_ni+n_mn]:
    # nmc_structure[i].symbol = "Mn"
    nmc_structure.replace(i, "Mn")
for i in ni_indices[n_ni+n_mn:n_ni+n_mn + n_co]:
    # nmc_structure[i].symbol = "Co"
    nmc_structure.replace(i, "Co")

print(f"NMC ratio: {nmc_ratio}")
# nmc_structure
print(f"n_ni:{n_ni}, n_mn:{n_mn}, n_co:{n_co}")

In [ ]:
nmc_structure

In [ ]:
save_cif(nmc_structure, f"{mp_id}_111")

In [ ]:
print(nmc_structure)

In [ ]:
sorted_sites = sorted(nmc_structure.sites, key=lambda site: Element(str(site.specie)).Z)

In [ ]:
structure_sorted = Structure.from_sites(sorted_sites)

In [ ]:
print(structure_sorted)

In [ ]:
# Write to POSCAR
Poscar(structure_sorted).write_file(f"POSCAR_{mp_id}_111_sorted")

Read triamine

In [ ]:
from pymatgen.io.xyz import XYZ

In [ ]:
# 2. Load triamine molecule from .xyz
with open("fg/triamine.xyz","r") as f:
    lines = XYZ.from_file(f).molecule

In [ ]:
from pymatgen.io.cif import CifParser

In [ ]:
poscar_structure = nmc_structure.get_structures()[0]

# Sort sites by atomic number (i.e., periodic table order)
sorted_sites = sorted(poscar_structure.sites, key=lambda site: Element(site.specie.symbol).Z)
poscar_structure_sorted = Structure.from_sites(sorted_sites)

# Write to POSCAR
poscar = Poscar(poscar_structure_sorted)
poscar.write_file("POSCAR_532")

In [ ]:
# Sort sites by atomic number (i.e., periodic table order)
sorted_sites = sorted(structure.sites, key=lambda site: Element(site.specie.symbol).Z)
structure_sorted = Structure.from_sites(sorted_sites)

# Write to POSCAR
poscar = Poscar(structure_sorted)
poscar.write_file("POSCAR_532")

In [ ]:
for i in ni_indices[n_ni:n_ni+n_mn]:
    print(nmc_structure[i])
# for i in ni_indices[n_ni+n_mn:n_ni+n_mn + n_co]:
#     print(i)

In [ ]:
for i, site in enumerate(nmc_structure):
    if site.specie.symbol == "Ni":
        print(i)

In [ ]:
print(nmc_structure)

In [ ]:
# Define compositions
nmc_compositions = {
    "111": (1/3, 1/3, 1/3),
    "532": (5/10, 3/10, 2/10)
    # "523": (5/10, 2/10, 3/10),
    # "910": (9/11, 1.5/11, 0.5/11),
    # "901": (9/11, 0.5/11, 1.5/11)
}

In [ ]:
# Loop through each and write a structure
for label, ratio in nmc_compositions.items():
    modified = substitute_Ni_atoms(structure_supercell_bottom.copy(), ratio, seed=42)
    # write(f"LiNMC_{label}_slab_sc.xyz", modified)
    # print(f"{label}: {Counter(atom.symbol for atom in modified)}")
    modified.to(filename=f"LiNMC_{label}_slab_sc.cif")
    print(f"{label}: {modified.composition}")

In [ ]:
ni_indices = [i for i, site in enumerate(structure_supercell_bottom) if site.specie.symbol == "Ni"]
print(ni_indices)

In [ ]:
# with MPRester(api_key) as mpr:
#     # thermo_docs = mpr.materials.thermo.search(material_ids=[mp_id])
#     thermo_doc = mpr.thermo.get_data_by_material_id(mp_id)

#     # # In addition, you can specify the level of theory by using "thermo_type", the default is "GGA_GGA+U_R2SCAN":
#     # thermo_docs = mpr.materials.thermo.search(
#     #     material_ids=[mp_id], thermo_types=["GGA_GGA+U", "GGA_GGA+U_R2SCAN", "R2SCAN"]
#     # )